Подлючение необходимых библиотек.

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
import numpy as np
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import BDeuScore, K2Score, BicScore
from pgmpy.estimators import PC,HillClimbSearch
from pomegranate import *
import seaborn as sns
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import *
from sklearn.preprocessing import KBinsDiscretizer
import time

<h2>Подготовка данных</h2>

Считывание изначального датасета.

In [ ]:
stud_data = pd.read_csv('dataset.csv')
stud_data.head()

Разделение датасета на тренировочнуд и тестовую выборки.

In [ ]:
x_train, x_test= sklearn.model_selection.train_test_split(stud_data, test_size=0.13, random_state=42)

Вывод тренировочной выборки.

In [ ]:
print(x_train)

Вывод тестовой выборки.

In [ ]:
print(x_test)

Преобразование категориальных параметров в числовые.

In [ ]:
for column in ['Name', 'Teacher']:
     lee = preprocessing.LabelEncoder()
     x_train[column] = lee.fit_transform(x_train[column].astype(str))

x_train.head()

Дискретизация непрерывных значений тренировочной выборки.

In [ ]:
trainsformed_data = x_train.copy()
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')
# data_discrete = est.fit_transform(x_test.values[:,[1,2,3,4,5,6,7,8,9,11,12]])
# testformed_data[["Danger", "Magician level", "Remarks", "Scores", "Absenses", "Lupin coef", "Quidditch", "Disease", "Beans", "StickLevel", "Volan-De-Mort"]] = data_discrete
data_discrete = est.fit_transform(x_train.values[:,[6,8]])
trainsformed_data[["Lupin coef", "Disease"]] = data_discrete

In [ ]:
trainsformed_data.head()

Подготовка выборки с пропущенными значениями (тестовая выборка).

Дискретизация непрерывных значений тестовой выборки.

In [ ]:
testformed_data = x_test.copy()
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')
# data_discrete = est.fit_transform(x_test.values[:,[1,2,3,4,5,6,7,8,9,11,12]])
# testformed_data[["Danger", "Magician level", "Remarks", "Scores", "Absenses", "Lupin coef", "Quidditch", "Disease", "Beans", "StickLevel", "Volan-De-Mort"]] = data_discrete
data_discrete = est.fit_transform(x_test.values[:,[6,8]])
testformed_data[["Lupin coef", "Disease"]] = data_discrete

In [ ]:
testformed_data.head()

Преобразование категориального параметра тестовой выборки в числовой.

In [ ]:
leon = 0
for column in ['Name', 'Teacher']:
     le = preprocessing.LabelEncoder()
     if column == 'Name':
          leon = le

     testformed_data[column] = le.fit_transform(testformed_data[column].astype(str))

testformed_data.head()

In [ ]:
mis_data = testformed_data.drop(columns=['Disease'])
mis_data.head()

Основными параметрами выборок были выбраны:

1.	Фамилия дежурного преподавателя (категориальный параметр).
2.	Имя студента. Выбирается студент, замеченный профессором ночью в коридоре. Были выбраны завсегдатые ночных прогулок: Гарри, Рон, Гермиона, Фред, Джордж, Джинни, Седрик, Драко, Краббе, Гойл (категориальный параметр).
3.	Уровень опасности, угрожающей студенту (дискретный числовой параметр) [Имя студента – родительский параметр].
4.	Уровень владения волшебством (измеряется в количестве выученных заклинаний, рецептов зелий) (дискретный числовой параметр) [Уровень волшебной палочки студента – родительский параметр].
5.	Количество замечаний от профессоров (дискретный числовой параметр) [Деж. преподаватель – родительский параметр].
6.	Количество очков, заработанных за день (дискретный числовой параметр) [Количество бобов у студентов – родительский параметр].
7.	Количество пропущенных занятий (дискретный числовой параметр).
8.	Коэффициент Люпина, который придумал профессор Люпин для оценки эффективности работы студентов, зависящий от пропущенных занятий, уровня владения волшебством, заработанных очков и уровня опасности (непрерывный числовой параметр) 
9.	Количество тренировок в квидич. Если студент является одной из команд в квидич, то количество тренировок равно половине пропущенных занятий (дискретный числовой параметр) [Деж. преподаватель – родительский параметр].
10.	 Вероятность получения травмы, заболеть (дискретный числовой параметр) [Количество тренировок в квидич, деятельность Волан-Де-Морта, уровень опасности, угрожающей студенту – родительские параметры].
11.	 Количество бобов у студентов (дискретный числовой параметр). [Имя студента – родительский параметр].
12.	 Уровень волшебной палочки студента (дискретный числовой параметр).
13.	 Деятельность Волан-Де-Морта (количество статей в газетах в день) (дискретный числовой параметр).

<h2>Построение корреляционной матрицы</h2>

In [ ]:
#отбор столбцов для построения корреляционной матрицы
x_train = trainsformed_data

#построение диагональной маски
mask = np.zeros_like(x_train.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(10, 10))

sns.heatmap(x_train.corr(), mask = mask, annot=True, fmt= '.1f', ax = ax, cmap = 'Blues')

<h2>Hill Climb Search</h2>

В сравнении метода hill climbing search исользуются методы оценки:

<h3>K2 Score</h3>

In [ ]:
hc1 = HillClimbSearch(trainsformed_data, scoring_method=K2Score(trainsformed_data))

<h3>BDeu Score</h3>

In [ ]:
hc2 = HillClimbSearch(trainsformed_data, scoring_method=BDeuScore(trainsformed_data))

<h3>Bic Score</h3>

In [ ]:
hc3 = HillClimbSearch(trainsformed_data, scoring_method=BicScore(trainsformed_data))

<h2>Оценка структуры данных</h2>

In [ ]:
tic = time.time()
model1 = hc1.estimate()
print('\n Время выполнения: ', time.time()-tic)

In [ ]:
tic = time.time()
model2 = hc2.estimate()
print('\n Время выполнения: ', time.time()-tic)

In [ ]:
tic = time.time()
model3 = hc3.estimate()
print('\n Время выполнения: ', time.time()-tic)

<h2>Проверка построенных структур данных</h2>

Как выяснилось, они одинаковые :)
<h2>Должны быть разные</h2>

In [ ]:
G_K1 = nx.DiGraph()
G_K1.add_edges_from(model1.edges())
pos = nx.layout.circular_layout(G_K1)
nx.draw(G_K1, pos, node_size=10, font_size=9, with_labels=True,font_weight='bold')

In [ ]:
G_K2 = nx.DiGraph()
G_K2.add_edges_from(model2.edges())
pos = nx.layout.circular_layout(G_K2)
nx.draw(G_K2, pos, node_size=10, font_size=9, with_labels=True,font_weight='bold')

In [ ]:
G_K3 = nx.DiGraph()
G_K3.add_edges_from(model3.edges())
pos = nx.layout.circular_layout(G_K3)
nx.draw(G_K3, pos, node_size=3, font_size=9, with_labels=True,font_weight='bold')

In [ ]:
# Создание байесовской модели.
bn1 = BayesianModel(model1.edges())

In [ ]:
# Создание байесовской модели.
bn2 = BayesianModel(model2.edges())

In [ ]:
# Создание байесовской модели.
bn3 = BayesianModel(model3.edges())

<h2>Обучение байесовских сетей</h2>

In [ ]:
tic = time.time()
# Обучение байесовской сети.
bn1.fit(trainsformed_data)
print('\n Время выполнения: ', time.time()-tic)

In [ ]:
tic = time.time()
# Обучение байесовской сети.
bn2.fit(trainsformed_data)
print('\n Время выполнения: ', time.time()-tic)

In [ ]:
tic = time.time()
# Обучение байесовской сети.
bn3.fit(trainsformed_data)
print('\n Время выполнения: ', time.time()-tic)

<h2>Заполнение пропусков</h2>

In [ ]:
print(bn1.nodes)

In [ ]:
tic = time.time()
# Заполнение пропусков
result_data1 = bn1.predict(mis_data)
print('\n Время выполнения: ', time.time()-tic)

In [ ]:
tic = time.time()
# Заполнение пропусков
result_data2 = bn2.predict(mis_data)
print('\n Время выполнения: ', time.time()-tic)

In [ ]:
tic = time.time()
# Заполнение пропусков
result_data3 = bn3.predict(mis_data)
print('\n Время выполнения: ', time.time()-tic)

<h2>Проверка предсказанных значений</h2>

In [ ]:
result_data1.head()

In [ ]:
result_data2.head()

In [ ]:
result_data3.head()

<h2>Подсчет точности заполнения пропусков сетями</h2>

In [ ]:
result_data1.reset_index(drop=True, inplace=True)
testformed_data.reset_index(drop=True, inplace=True)
concat_data = result_data1[['Disease']]
concat_data = pd.concat([testformed_data[['Lupin coef']], concat_data],axis=1)

In [ ]:
concat_data.head()

In [ ]:
pair = est.inverse_transform(concat_data)

In [ ]:
print(pair)

In [ ]:
import math
error = 0
for i in range(len(pair)):
    print(pair[i,0], x_test.values[i,8])
    error += (pair[i,0]-x_test.values[i,8])**2

error = math.sqrt(error / len(pair))
print('\n Средневкадратическая ошибка: ', error)

In [ ]:
result_data2.reset_index(drop=True, inplace=True)
testformed_data.reset_index(drop=True, inplace=True)
concat_data = result_data2[['Disease']]
concat_data = pd.concat([testformed_data[['Lupin coef']], concat_data],axis=1)

In [ ]:
concat_data.head()

In [ ]:
porosity = est.inverse_transform(concat_data)

In [ ]:
print(porosity)

In [ ]:
import math
error = 0
for i in range(len(porosity)):
    print(porosity[i,1], x_test.values[i,8])
    error += (porosity[i,1]-x_test.values[i,8])**2

error = math.sqrt(error / len(porosity))
print('\n Средневкадратическая ошибка: ', error)

In [ ]:
result_data3.reset_index(drop=True, inplace=True)
testformed_data.reset_index(drop=True, inplace=True)
concat_data = result_data3[['Disease']]
concat_data = pd.concat([testformed_data[['Lupin coef']], concat_data],axis=1)

In [ ]:
porosity = est.inverse_transform(concat_data)

In [ ]:
import math
error = 0
for i in range(len(porosity)):
    print(porosity[i,1], x_test.values[i,8])
    error += (porosity[i,1]-x_test.values[i,8])**2

error = math.sqrt(error / len(porosity))
print('\n Средневкадратическая ошибка: ', error)

In [ ]:
regime1 = leon.inverse_transform(result_data1.values.astype(int))

In [ ]:
regime2 = leon.inverse_transform(result_data2.values.astype(int))

In [ ]:
regime3 = leun.inverse_transform(result_data3.values[:,1].astype(int))

In [ ]:
def true_false(mis_data: pd.DataFrame, regime: pd.DataFrame):
    true, false = 0, 0

    for i in range(len(regime)):
        print(regime[i], mis_data.values[i,0], regime[i] in mis_data.values[i,0])
        if regime[i] in mis_data.values[i,0]:
            true += 1
        else:
            false += 1
    
    print('Число совпадений: ', true)
    print('Число промахов: ', false)
    
    sns.barplot(x=['True','False'], y=[true, false])
    plt.show()

In [ ]:
true_false(x_test,regime1)

In [ ]:
true_false(x_test,regime2)

In [ ]:
true_false(mis_data,regime3)

<h2>Нахождение среднего, максимального и минимального времен построения модели</h2>

In [ ]:
# md_times = []
# max_times = []
# min_times = []
# bn_times = []

# for i in range(3):
#      bn_times.append([])

# for i in range(100):
#      tic = time.time()
#      bn1.fit(trainsformed_data)
#      bn_times[0].append(time.time() - tic)
#      tic = time.time()
#      bn2.fit(trainsformed_data)
#      bn_times[1].append(time.time() - tic)
#      tic = time.time()
#      bn3.fit(trainsformed_data)
#      bn_times[2].append(time.time() - tic)

# for i in range(3):
#      max_times.append(max(bn_times[i]))
#      min_times.append(min(bn_times[i]))
#      md_times.append(sum(bn_times[i])/len(bn_times[i]))

In [ ]:
# def give_barplot(var):
#      sns.barplot(x=['K2Score','BDeuScore','BicScore'], y=var)
#      plt.show()

In [ ]:
# give_barplot(max_times)

In [ ]:
# give_barplot(min_times)

In [ ]:
# give_barplot(md_times)

<h2>PC algorithm</h2>

Не работает, потому что при построении модели куда-то теряется параметр :(

<h2>Создание модели</h2>.

In [ ]:
pc = PC(trainsformed_data)

<h2>Оценка структуры данных</h2>

In [ ]:
tic = time.time()
model = pc.estimate()
print('\n Время выполнения: ', time.time()-tic)

In [ ]:
G_K2 = nx.DiGraph()
G_K2.add_edges_from(model.edges())
pos = nx.layout.circular_layout(G_K2)
nx.draw(G_K2, pos, node_size=10, font_size=9, with_labels=True,font_weight='bold')

<h2>Создание байесовской сети</h2>

In [ ]:
# Создание байесовской модели.
bn = BayesianModel(model.edges())

<h2>Обучение сетей</h2>

In [ ]:
tic = time.time()
# Обучение параметров.
bn.fit(trainsformed_data)
print('\n Время выполнения: ', time.time()-tic)

In [ ]:
tic = time.time()
# Заполнение пропусков
result_data = bn.predict(mis_data)
print('\n Время выполнения: ', time.time()-tic)

<h2>Подсчет точности заполнения пропусков сетями</h2>

In [ ]:
result_data.reset_index(drop=True, inplace=True)
testformed_data.reset_index(drop=True, inplace=True)
concat_data = result_data[['Disease']]
concat_data = pd.concat([testformed_data[['Lupin coef']], concat_data],axis=1)

In [ ]:
porosity = est.inverse_transform(concat_data)

In [ ]:
import math
error = 0
for i in range(len(porosity)):
    print(porosity[i,1], x_test.values[i,8])
    error += (porosity[i,1]-x_test.values[i,8])**2

error = math.sqrt(error / len(porosity))
print('\n Средневкадратическая ошибка: ', error)

In [ ]:
regime = leon.inverse_transform(result_data.values.astype(int))

In [ ]:
true_false(x_test,regime)